## dbt Cloud API Example for Auto Updating Webhooks for to contain all jobs within a dbt Cloud Project (or environment within a project)
- This can be done via scripting with the dbt Cloud APIs or VIA the [dbt Cloud Terraform Provider](https://registry.terraform.io/providers/dbt-labs/dbtcloud/latest)

In [1]:
import requests
from datetime import datetime
import json

In [3]:
# required inputs
dbt_cloud_api_token = "dbtc_12341241243124"
dbt_cloud_account_id = 12345
dbt_cloud_project_id = 987654
dbt_cloud_webhook_id = 'wsu_2n5GxuzXauxkEHsWdSbsyfr5S1e'

# optional inputs, only use if you want to filter for only jobs within an environment in a given project
# dbt_cloud_environment_id = 654321

___

#### __Step 1:__ Function to get all jobs within a given project (or project and environment)

In [7]:
def get_dbt_cloud_job_ids(api_token, account_id, project_id, environment_id=None):
    """
    Get all job IDs for a specified project in dbt Cloud, with an optional environment filter.

    Parameters:
        api_token (str): The API token for authenticating with dbt Cloud.
        account_id (int): The account ID in dbt Cloud.
        project_id (int): The project ID for which to fetch job IDs.
        environment_id (int, optional): The environment ID to filter jobs. Defaults to None.

    Returns:
        list: A list of job IDs within the specified project and optional environment.
    """
    url = f"https://cloud.getdbt.com/api/v2/accounts/{account_id}/jobs/"
    headers = {
        "Authorization": f"Token {api_token}"
    }
    
    # Make the request to get all jobs within the account
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        jobs = response.json().get('data', [])
        # Filter jobs by project_id and optionally by environment_id
        job_ids = [
            job['id'] for job in jobs 
            if job['project_id'] == project_id and 
               (environment_id is None or job['environment_id'] == environment_id)
        ]
        return job_ids
    else:
        print("Failed to retrieve jobs:", response.status_code, response.text)
        return []

#### __Step 2:__ Function to update a given webhook to apply to all of the given jobs within a project

In [10]:
def retrieve_and_update_webhook_subscription(api_token, account_id, webhook_id, updated_job_ids_for_webhook):
    """
    Retrieve a dbt Cloud webhook subscription and update it with the retrieved payload using the v3 API.

    Parameters:
        api_token (str): The API token for authenticating with dbt Cloud.
        account_id (int): The account ID in dbt Cloud.
        webhook_id (int): The webhook subscription ID to retrieve and update.

    Returns:
        dict: The updated webhook subscription details if successful, else an error message.
    """
    url = f"https://cloud.getdbt.com/api/v3/accounts/{account_id}/webhooks/subscription/{webhook_id}"
    headers = {
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json"
    }
    
    # Step 1: Retrieve the current webhook subscription details
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print("Failed to retrieve webhook subscription:", response.status_code, response.text)
        return {"error": response.text}
    
    webhook_data = response.json().get('data', {})

    # create the webhook payload with the new job ids
    update_webhook_payload = {
        "id": webhook_data['id'],
        "name": webhook_data['name'],
        "event_types": webhook_data['event_types'],
        "client_url": webhook_data['client_url'],
        "active": webhook_data['active'],
        "job_ids": updated_job_ids_for_webhook

    }
    
    # Step 2: Update the webhook subscription with the retrieved payload
    update_response = requests.put(url, headers=headers, json=update_webhook_payload)
    
    if update_response.status_code == 200:
        print("Webhook subscription updated successfully.")
        return update_response.json().get('data', {})
    else:
        print("Failed to update webhook subscription:", update_response.status_code, update_response.text)
        return {"error": update_response.text}

#### __Step 3:__ Putting it all together -- getting all of the job ids within a given project (or project + environment) and updating the webhook to apply to all said jobs

##### __Note:__ This operation is idempotent and can be applied over and over again and will only change if a new set of job ids is applied to it

In [11]:
# get the dbt Cloud job ids for a given project
dbt_cloud_job_ids = get_dbt_cloud_job_ids(dbt_cloud_api_token, dbt_cloud_account_id, dbt_cloud_project_id)

# logging the job ids
print(f"The dbt Cloud Job IDs for the selected project are: {dbt_cloud_job_ids} ")

# updating the webhook to apply to all jobs from above
updated_webhook = retrieve_and_update_webhook_subscription(dbt_cloud_api_token, dbt_cloud_account_id, dbt_cloud_webhook_id, dbt_cloud_job_ids)

# logging the update has happened
print(f"The dbt Cloud Webhook has been updated and now apply to the following Job IDs: {updated_webhook['job_ids']}")

The dbt Cloud Job IDs for the selected project are: [106102, 211981, 494432, 494574, 548096, 644841, 687105] 
Webhook subscription updated successfully.
The dbt Cloud Webhook has been updated and now apply to the following Job IDs: ['106102', '211981', '494432', '494574', '548096', '644841', '687105']


##### __Example if filtering for environment as well__

In [13]:
# further filter down the job ids by providing an environment_id
dbt_cloud_environment_id = 654321

# get the dbt Cloud job ids for a given project + environment
dbt_cloud_job_ids = get_dbt_cloud_job_ids(dbt_cloud_api_token, dbt_cloud_account_id, dbt_cloud_project_id, dbt_cloud_environment_id)

# logging the job ids
print(f"The dbt Cloud Job IDs for the selected project + environment are: {dbt_cloud_job_ids} ")

# updating the webhook to apply to all jobs from above
updated_webhook = retrieve_and_update_webhook_subscription(dbt_cloud_api_token, dbt_cloud_account_id, dbt_cloud_webhook_id, dbt_cloud_job_ids)

# logging the update has happened
print(f"The dbt Cloud Webhook has been updated and now apply to the following Job IDs: {updated_webhook['job_ids']}")

The dbt Cloud Job IDs for the selected project + environment are: [106102, 494432, 494574, 548096, 644841] 
Webhook subscription updated successfully.
The dbt Cloud Webhook has been updated and now apply to the following Job IDs: ['106102', '494432', '494574', '548096', '644841']


# END OF SCRIPT